In [0]:

match_df = spark.read.parquet("dbfs:/FileStore/ipl_final_cleaned.parquet")
match_df.printSchema()
match_df.show(5)



root
 |-- year: integer (nullable = true)
 |-- series_type: timestamp (nullable = true)
 |-- series_name: string (nullable = true)
 |-- match_no: string (nullable = true)
 |-- match_type: string (nullable = true)
 |-- match_name: string (nullable = true)
 |-- match_href: string (nullable = true)
 |-- match_team1: string (nullable = true)
 |-- match_team2: string (nullable = true)
 |-- match_datetime_start: timestamp (nullable = true)
 |-- match_date_end: string (nullable = true)
 |-- match_venue: string (nullable = true)

+----+-------------------+--------------------+--------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------+-----------+
|year|        series_type|         series_name|match_no|match_type|          match_name|          match_href|         match_team1|         match_team2|match_datetime_start|match_date_end|match_venue|
+----+-------------------+--------------------+--------+----------+-----

In [0]:
from pyspark.sql.functions import col, regexp_extract, lit, sum as _sum

data = [
    ("Bumrah to Dhawan, FOUR! Cracking shot through covers", "Bumrah", "Dhawan", "FOUR"),
    ("Chahal to Raina, 1 run, nudged to midwicket", "Chahal", "Raina", "1 run"),
    ("Narine to Kohli, no run, defended solidly", "Narine", "Kohli", "no run"),
    ("Rabada to Rohit, SIX! Smashed over long-on", "Rabada", "Rohit", "SIX"),
]

ball_df = spark.createDataFrame(data, ["commentary", "bowler", "batsman", "runs_raw"])

ball_df = ball_df.withColumn(
    "runs",
    regexp_extract("runs_raw", r"(\d+)", 1).cast("int")
).fillna(0)

ball_df = ball_df.withColumn("match_id", col("bowler")) \
                 .withColumn("team", col("bowler")) \
                 .withColumn("year", lit(2021))

ball_df.show()



+--------------------+------+-------+--------+----+--------+------+----+
|          commentary|bowler|batsman|runs_raw|runs|match_id|  team|year|
+--------------------+------+-------+--------+----+--------+------+----+
|Bumrah to Dhawan,...|Bumrah| Dhawan|    FOUR|   0|  Bumrah|Bumrah|2021|
|Chahal to Raina, ...|Chahal|  Raina|   1 run|   1|  Chahal|Chahal|2021|
|Narine to Kohli, ...|Narine|  Kohli|  no run|   0|  Narine|Narine|2021|
|Rabada to Rohit, ...|Rabada|  Rohit|     SIX|   0|  Rabada|Rabada|2021|
+--------------------+------+-------+--------+----+--------+------+----+



In [0]:
team_scores_df = ball_df.groupBy("year", "match_id", "team") \
                        .agg(_sum("runs").alias("total_score"))

team_scores_df.show()


+----+--------+------+-----------+
|year|match_id|  team|total_score|
+----+--------+------+-----------+
|2021|  Bumrah|Bumrah|          0|
|2021|  Chahal|Chahal|          1|
|2021|  Narine|Narine|          0|
|2021|  Rabada|Rabada|          0|
+----+--------+------+-----------+



In [0]:
avg_score_pivot = team_scores_df.groupBy("team") \
    .pivot("year") \
    .avg("total_score") \
    .orderBy("team")

avg_score_pivot.show()


+------+----+
|  team|2021|
+------+----+
|Bumrah| 0.0|
|Chahal| 1.0|
|Narine| 0.0|
|Rabada| 0.0|
+------+----+



In [0]:
max_score_pivot = team_scores_df.groupBy("team") \
    .pivot("year") \
    .max("total_score") \
    .orderBy("team")

max_score_pivot.show()


+------+----+
|  team|2021|
+------+----+
|Bumrah|   0|
|Chahal|   1|
|Narine|   0|
|Rabada|   0|
+------+----+

